In [2]:
import numpy as np
import pandas as pd

# Lab 9 - Multi-layer Perceptron Forward Pass & Backpropagation

## Part I
For this exercise you will implement a simple 2-layer perceptron with the forward pass and the backpropagation to learn the weights

For the first part you'll build and train a 2-layer neural network that predicts the prices of houses, using the usual Boston housing dataset.

In [3]:
boston = pd.read_csv('data/BostonHousing.txt')
boston.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


As usual, consider the MEDV as your target variable. 
* Split the data into training, validation and testing (70,15,15)%
* Experiment with different number of neurons per layer for your network, using the validation set

In [4]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
        self.output_without_activation = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [5]:
# activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def identity(x):
    return x

def identity_prime(x):
    return 1

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [6]:
# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size, activation, activation_prime):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.activation = activation
        self.activation_prime = activation_prime

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output_without_activation = np.dot(self.input, self.weights) + self.bias
        self.output = self.activation(self.output_without_activation)
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        output_error = self.activation_prime(self.output_without_activation) * output_error
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [7]:

class FCLayer():
    def __init__(self, input_size, output_size, activation, activation_prime):
        self.input = None
        self.output = None
        self.output_without_activation = None
        self.weights = np.random.rand(input_size, output_size) * 0.1
        self.bias = np.random.rand(1, output_size) * 0.1
        self.activation = activation
        self.activation_prime = activation_prime

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output_without_activation = np.dot(self.input, self.weights) + self.bias
        self.output = self.activation(self.output_without_activation)
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        output_error = self.activation_prime(self.output_without_activation) * output_error
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [8]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [9]:
# your code goes here
from sklearn.model_selection import train_test_split

X = boston.drop('medv', axis=1)
y = boston['medv']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.50, random_state=42)

x_train = X_train.values
y_train = y_train.values

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_normalized = np.array(scaler.fit_transform(x_train))

x_train = x_train.reshape(-1, 1, 13)
y_train = y_train.reshape(-1, 1, 1)
x_train_normalized = x_train_normalized.reshape(-1, 1, 13)

# network
net = Network()
net.add(FCLayer(13, 15, sigmoid, sigmoid_prime))
net.add(FCLayer(15, 1, identity, identity_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train_normalized, y_train, epochs=10000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/10000   error=123.595439
epoch 2/10000   error=59.582532
epoch 3/10000   error=66.329795
epoch 4/10000   error=51.885463
epoch 5/10000   error=58.011435
epoch 6/10000   error=44.067241
epoch 7/10000   error=35.027334
epoch 8/10000   error=26.896427
epoch 9/10000   error=30.405429
epoch 10/10000   error=34.074392
epoch 11/10000   error=26.154567
epoch 12/10000   error=26.252754
epoch 13/10000   error=30.907262
epoch 14/10000   error=30.383892
epoch 15/10000   error=27.379973
epoch 16/10000   error=30.628469
epoch 17/10000   error=31.057378
epoch 18/10000   error=33.614457
epoch 19/10000   error=31.703153
epoch 20/10000   error=28.714034
epoch 21/10000   error=22.235744
epoch 22/10000   error=22.535261
epoch 23/10000   error=21.654172
epoch 24/10000   error=18.902891
epoch 25/10000   error=22.695348
epoch 26/10000   error=21.079820
epoch 27/10000   error=19.837906
epoch 28/10000   error=18.798632
epoch 29/10000   error=18.080086
epoch 30/10000   error=19.265894
epoch 31/10000   e

/tmp/ipykernel_14971/1470148461.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


## Part II 

For this exercise you will build and train a 2-layer neural network that predicts the exact digit from a hand-written image, using the MNIST dataset. 
For this exercise, add weight decay to your network.

In [12]:
from sklearn.preprocessing import StandardScaler

X = boston.drop('medv', axis=1)
y = boston['medv']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.50, random_state=42)

x_train = X_train.values
y_train = y_train.values

scaler = StandardScaler()
x_train_normalized = np.array(scaler.fit_transform(x_train))

x_train_normalized

array([[ 4.18086885, -0.58340733,  1.11352941, ...,  0.94048457,
        -3.76287569,  0.27718055],
       [-0.34572243, -0.58340733,  2.53014484, ...,  0.89468931,
        -0.43228216,  2.32707749],
       [ 0.75786506, -0.58340733,  1.11352941, ...,  0.94048457,
        -3.29709728,  1.56985962],
       ...,
       [-0.36477537,  0.3100929 , -0.71749013, ..., -0.61655426,
         0.44002409, -0.54629388],
       [-0.35160812, -0.58340733,  0.04225072, ...,  0.48253197,
         0.21814015, -0.35022854],
       [-0.36407441,  2.39492677, -1.21712628, ..., -1.53245945,
         0.15855609, -1.04524637]])

Again, you will split the data into training, validation and testing.

In [17]:
# your code goes here:
class MyStandardScaler:
    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
    
    def transform(self, X):
        return (X - self.mean_) / self.scale_
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

# Example usage
scaler = MyStandardScaler()
x_train_normalized = scaler.fit_transform(x_train)
x_train_normalized

np.mean(X, axis=0)

crim         3.613524
zn          11.363636
indus       11.136779
chas         0.069170
nox          0.554695
rm           6.284634
age         68.574901
dis          3.795043
rad          9.549407
tax        408.237154
ptratio     18.455534
b          356.674032
lstat       12.653063
dtype: float64

In [ ]:
# your code goes here:
